In [1]:
import cv2
import numpy as np
from torchvision.datasets import CocoCaptions
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.transforms.functional import resize
import torchvision

## Unet

In [2]:
import Detection.GroundingDINO.groundingdino.datasets.transforms as T

transform = T.Compose(
        [
            T.RandomResize([800], max_size=1333),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )


In [3]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self,phase, transform):
        self.phase = phase
        
        if phase == 'train':
            self.dataset = CocoCaptions(root='./train2017', annFile='./annotations/captions_train2017.json')
        else:
            self.dataset = CocoCaptions(root='./val2017', annFile='./annotations/captions_val2017.json')
        
        self.transform = transform
        
        
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self,idx):
        
        org_img, caps = self.dataset[idx]

        img,_ = self.transform(org_img, None)
        
        random_idx = torch.randint(0,len(caps), (1,))
        
        if self.phase == 'train':
            return np.array(org_img), img, caps[random_idx]
        else:
            return np.array(org_img), img, caps[1]


In [4]:
from SResolution.TG_Umodels import *

def select_model(down_scale, shape):
    if down_scale == 2:
        model = TG_UNet2(3,3, shape)
    elif down_scale == 4:
        model = TG_UNet4(3,3, shape)
    elif down_scale == 8:
        model = TG_UNet8x2(3,3, shape)

    return model

In [5]:
class VGGPerceptualLoss(torch.nn.Module):
    def __init__(self, resize=True):
        super(VGGPerceptualLoss, self).__init__()
        blocks = []
        blocks.append(torchvision.models.vgg16(pretrained=True).features[:4].eval())
        blocks.append(torchvision.models.vgg16(pretrained=True).features[4:9].eval())
        blocks.append(torchvision.models.vgg16(pretrained=True).features[9:16].eval())
        blocks.append(torchvision.models.vgg16(pretrained=True).features[16:23].eval())
        for bl in blocks:
            for p in bl.parameters():
                p.requires_grad = False
        self.blocks = torch.nn.ModuleList(blocks)
        self.transform = torch.nn.functional.interpolate
        self.resize = resize
        self.register_buffer("mean", torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1))
        self.register_buffer("std", torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1))

    def forward(self, input, target, feature_layers=[0, 1, 2, 3], style_layers=[]):
        if input.shape[1] != 3:
            input = input.repeat(1, 3, 1, 1)
            target = target.repeat(1, 3, 1, 1)
        input = (input-self.mean) / self.std
        target = (target-self.mean) / self.std
        if self.resize:
            input = self.transform(input, mode='bilinear', size=(224, 224), align_corners=False)
            target = self.transform(target, mode='bilinear', size=(224, 224), align_corners=False)
        loss = 0.0
        x = input
        y = target
        for i, block in enumerate(self.blocks):
            x = block(x)
            y = block(y)
            if i in feature_layers:
                loss += torch.nn.functional.l1_loss(x, y)
            if i in style_layers:
                act_x = x.reshape(x.shape[0], x.shape[1], -1)
                act_y = y.reshape(y.shape[0], y.shape[1], -1)
                gram_x = act_x @ act_x.permute(0, 2, 1)
                gram_y = act_y @ act_y.permute(0, 2, 1)
                loss += torch.nn.functional.l1_loss(gram_x, gram_y)
        return loss

In [6]:


down_scale = 8
input_shape = (128,128)
model = select_model(down_scale,input_shape)
patience = 5
L2 = nn.MSELoss()
L1 = nn.L1Loss()
per_loss = VGGPerceptualLoss()
lam = 0.6
epochs = 200
lr = 1e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
per_loss = per_loss.to(device)


print(device)


c:\Users\juniy\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\juniy\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda:0


In [7]:
from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler, DataLoader, Subset

trainDataset = CustomDataset('train', transform)

num_train_samples = 1000
sample_ds = Subset(trainDataset, np.arange(num_train_samples))
sample_sampler = RandomSampler(sample_ds)
# sample_dl = DataLoader(sample_ds, sampler=sample_sampler, batch_size=1)
train_dataloader = DataLoader(sample_ds, batch_size=1, shuffle=True)

valDataset = CustomDataset('val', transform)
num_train_samples = 200
sample_ds = Subset(valDataset, np.arange(num_train_samples))
sample_sampler = RandomSampler(sample_ds)
val_dataloader = DataLoader(sample_ds, batch_size=1, shuffle=False)

loading annotations into memory...
Done (t=0.50s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [8]:
print(len(train_dataloader), len(val_dataloader))

1000 200


In [9]:
oo, a, c = next(iter(train_dataloader))

oo.shape, a.shape, type(c)

(torch.Size([1, 427, 640, 3]), torch.Size([1, 3, 800, 1199]), tuple)

In [12]:
from Detection.GroundingDINO.groundingdino.util.inference import load_model, load_image_6, predict, annotate
from tqdm import tqdm 
import time
from torchvision.ops import box_convert
from torchvision.transforms.functional import center_crop
from copy import deepcopy
from torchvision.transforms import InterpolationMode


iterable = range(epochs)

dect_model = load_model("./Detection/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py", "./Detection/GroundingDINO//weights/groundingdino_swint_ogc.pth")
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25

target_size = 128

model.to(device)
send_tg_message(f'Start Server Computer X8')
best_loss = float("inf") 
best_result = None
scheduler_cnt = 0
lr_list = [lr]
for e in iterable:
    for p in ['Train', 'Test']:
        if p == 'Train':
            total_loss = 0
            total = 0
            model.train()

            train_result = {}
            
            for oo, bimg, caps in tqdm(train_dataloader, desc='Train', leave=False):
                # print(f'{str(caps[0])}')
                train_result[f'{str(caps[0])}'] = {
                    'phrases': [],
                    'org_img':[],
                    'shape':[],
                    'cropped':[],
                    'padded':[],
                    'resized':[],
                    'output':[],
                    'c_padded':[]
                }
                
                i_caps = str(caps[0])
                bimg = bimg.squeeze()
                b_boxes, b_logits, b_phrases, b_encoded_text = predict(
                        model=dect_model,
                        image=bimg,
                        caption=i_caps,
                        box_threshold=BOX_TRESHOLD,
                        text_threshold=TEXT_TRESHOLD
                    )
                train_result[f'{str(caps[0])}']['phrases'].append(b_phrases)
                oo = oo.squeeze()
                h, w, c = oo.shape
                boxes2 = b_boxes * torch.Tensor([w, h, w, h])
                xyxy = box_convert(boxes=boxes2, in_fmt="cxcywh", out_fmt="xyxy").numpy()
                
                for xyxy_idx in range(len(xyxy)):
                    x1, y1, x2, y2 = xyxy[xyxy_idx].astype(int)
                    cropped_img = oo[y1:y2, x1:x2].permute(2,1,0)

                    _, h, w = cropped_img.shape
                    if h>w:
                        if h>target_size:
                            cropped_img = resize(cropped_img, (target_size, int(target_size*(w/h))))
                    else:
                        if w>target_size:
                            cropped_img = resize(cropped_img, (int(target_size*(h/w)), target_size))


                    train_result[f'{str(caps[0])}']['cropped'].append(cropped_img)
                    _, h, w = cropped_img.shape
                    train_result[f'{str(caps[0])}']['shape'].append((h,w))

                    # half_crop = resize(cropped_img, (h//down_scale, w//down_scale))
                    cc, hh, ww = cropped_img.shape
                    # cc, hh, ww = half_crop.shape
                    nh, nw = target_size - hh,  target_size - ww
                    padd = (nw//2, nw//2, nh//2, nh//2)
                    s_img = torch.nn.functional.pad(cropped_img, padd)
                    s_img = resize(s_img, (target_size, target_size))
                    train_result[f'{str(caps[0])}']['resized'].append(s_img)
                

                txt = b_encoded_text['encoded_text']


                
                
                for re, pa, sha in zip(train_result[f'{str(caps[0])}']['resized'], train_result[str(caps[0])]['cropped'],train_result[str(caps[0])]['shape'] ):
                    re, pa = re/255., pa/255.
                    # re, pa = re.float(), pa.float()
                    
                    re = re.unsqueeze(0)
                    # pa = pa.unsqueeze(0)
                    re = re.to(device)
                    # pa = pa.to(device)

                    pa = torch.tensor(pa).unsqueeze(0)
                    pa = pa.to(device)

                    output = model(re,txt)
                    h,w = sha
                    c_output = center_crop(output, (h*down_scale, w*down_scale))
                    # c_pa = center_crop(pa, sha)
                    # print(c_output.shape, pa.shape)
                    for_loss = resize(c_output, (h, w), interpolation=InterpolationMode.BICUBIC)

                    # loss = criterion(c_output, c_pa)
                    l2_loss = L2(for_loss, pa)
                    l1_loss = L1(for_loss, pa)
                    p_loss = per_loss(for_loss, pa)

                    loss = lam*p_loss+(1-lam/2)*l1_loss+(1-lam/2)*l2_loss

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    total_loss += loss
                    total += 1
            avg_loss = total_loss / total
            train_text = f'Train | Epoch: [{e+1}/{epochs}] |  MSE: {avg_loss} | Total: {total}'
            print('='*len(train_text))
            print(train_text)

        else:
            total_loss = 0
            total = 0
            model.eval()

            test_result = {}
            
            with torch.no_grad():
                for oo, bimg, caps in tqdm(val_dataloader, desc='Test', leave=False):

                    test_result[f'{str(caps[0])}'] = {
                        'phrases': [],
                        'org_img':[],
                        'shape':[],
                        'cropped':[],
                        'padded':[],
                        'resized':[],
                        'output':[],
                        'c_padded':[],
                        'detect':[],
                        'for_loss':[]
                    }

                    i_caps = str(caps[0])
                    bimg = bimg.squeeze()
                    b_boxes, b_logits, b_phrases, b_encoded_text = predict(
                            model=dect_model,
                            image=bimg,
                            caption=i_caps,
                            box_threshold=BOX_TRESHOLD,
                            text_threshold=TEXT_TRESHOLD
                        )
                    test_result[f'{str(caps[0])}']['phrases'].append(b_phrases)
                    oo = oo.squeeze()
                    h, w, c = oo.shape
                    boxes2 = b_boxes * torch.Tensor([w, h, w, h])
                    xyxy = box_convert(boxes=boxes2, in_fmt="cxcywh", out_fmt="xyxy").numpy()
                    test_result[f'{str(caps[0])}']['org_img'].append(oo)

                    for xyxy_idx in range(len(xyxy)):
                        x1, y1, x2, y2 = xyxy[xyxy_idx].astype(int)
                        cropped_img = oo[y1:y2, x1:x2].permute(2,1,0)

                        _, h, w = cropped_img.shape
                        if h>w:
                            if h>target_size:
                                cropped_img = resize(cropped_img, (target_size, int(target_size*(w/h))))
                        else:
                            if w>target_size:
                                cropped_img = resize(cropped_img, (int(target_size*(h/w)), target_size))
                        
                        test_result[f'{str(caps[0])}']['cropped'].append(cropped_img)
                        _, hh, ww = cropped_img.shape
                        test_result[f'{str(caps[0])}']['shape'].append((hh,ww))
                        
                        nh, nw = target_size - hh,  target_size - ww
                        padd = (nw//2, nw//2, nh//2, nh//2)
                        s_img = torch.nn.functional.pad(cropped_img, padd)
                        s_img = resize(s_img, (target_size, target_size))
                        test_result[f'{str(caps[0])}']['resized'].append(s_img)
                
                        

                    # for b in test_result[f'{str(caps[0])}']['padded']:
                    #     _, h, w = b.shape
                    #     s_img = resize(b, (h//down_scale, w//down_scale))
                        
                    #     test_result[f'{str(caps[0])}']['resized'].append(s_img)

                    txt = b_encoded_text['encoded_text']

                    for re, pa, sha in zip(test_result[f'{str(caps[0])}']['resized'], test_result[str(caps[0])]['cropped'], test_result[str(caps[0])]['shape']):
                        re, pa = re/255., pa/255.
                        # re, pa = re.float(), pa.float()
                        # re = re.unsqueeze(0)
                        # pa = pa.unsqueeze(0)
                        # re = re.to(device)
                        # pa = pa.to(device)
                        re = re.unsqueeze(0)
                        # pa = pa.unsqueeze(0)
                        re = re.to(device)
                        # pa = pa.to(device)

                        pa = torch.tensor(pa).unsqueeze(0)
                        pa = pa.to(device)

                        output = model(re,txt)
                        h, w = sha
                        c_output = center_crop(output, (h*down_scale, w*down_scale))
                        # c_pa = center_crop(pa, sha)
                        for_loss = resize(c_output, (h, w),interpolation=InterpolationMode.BICUBIC)
                        test_result[f'{str(caps[0])}']['output'].append(c_output.detach().squeeze(0).cpu().numpy())
                        test_result[f'{str(caps[0])}']['for_loss'].append(for_loss.detach().squeeze(0).cpu().numpy())
                        # test_result[f'{str(caps[0])}']['c_padded'].append(c_pa.detach().squeeze(0).cpu().numpy())
                        # test_result[f'{str(caps[0])}']['label'].append(pa.detach().squeeze(0).cpu().numpy())

                        l2_loss = L2(for_loss, pa)
                        l1_loss = L1(for_loss, pa)

                        p_loss = per_loss(for_loss, pa)
                        loss = lam*p_loss+(1-lam/2)*l1_loss+(1-lam/2)*l2_loss
                        total_loss += loss
                        total += 1
                avg_loss = total_loss / total
                test_text = f'Test  | Epoch: [{e+1}/{epochs}] |  MSE: {avg_loss} | Total: {total}'
                print(test_text)
                print('='*len(train_text))

                if avg_loss < best_loss:
                    print('Update results')
                    scheduler_cnt = 0
                    best_result = test_result
                    best_loss = avg_loss
                    best_model = deepcopy(model.state_dict())
                else:
                    scheduler_cnt += 1
                    print(scheduler_cnt)
                    if scheduler_cnt == patience:
                        optimizer.param_groups[0]['lr'] /= 2
                        lr_list.append(optimizer.param_groups[0]['lr'])
                        print(optimizer.param_groups[0]['lr'])
                        scheduler_cnt = 0
                        

send_tg_message(f'Finish Server Computer X8 - real inverse + L1 & L2 not softmax')

final text_encoder_type: bert-base-uncased


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Train | Epoch: [1/200] |  MSE: 1.0675249099731445 | Total: 2837


Test:   0%|          | 0/200 [00:00<?, ?it/s]UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Test  | Epoch: [1/200] |  MSE: 0.7643295526504517 | Total: 569
Update results


...

Test  | Epoch: [198/200] |  MSE: 0.17776690423488617 | Total: 569
Update results


Train | Epoch: [199/200] |  MSE: 0.17871354520320892 | Total: 2820


Test  | Epoch: [199/200] |  MSE: 0.1779383420944214 | Total: 569
1


Train | Epoch: [200/200] |  MSE: 0.18182936310768127 | Total: 2712


Test  | Epoch: [200/200] |  MSE: 0.17782944440841675 | Total: 569
2


In [17]:
import os
from PIL import Image
from torchvision.transforms.functional import center_crop

def save_imgs(results, root='results'):
    

    for k in results.keys():
        # root = './resultsX4'
        if not os.path.isdir(root):
            os.mkdir(root)

        if not os.path.isdir(os.path.join(root, k)):
            os.mkdir(os.path.join(root, k))
        save_path = os.path.join(root, k)
        
        org = Image.fromarray(results[k]['org_img'][0].numpy())
        org.save(os.path.join(save_path, 'org.png'))
        if len(results[k]['phrases'][0]) == 0:
            print('ups')
            continue
        zz = 0
        for i in range(len(results[k]['phrases'][0])):
            save_path = os.path.join(root, k)
            
            output = torch.tensor(results[k]['output'][i])
            output = Image.fromarray((np.clip(output.permute(2,1,0).numpy(),0,1)*255).astype(np.uint8))

            # noc_output = torch.tensor(results[k]['output'][i])
            # noc_output = Image.fromarray((np.clip(noc_output.permute(2,1,0).numpy(),0,1)*255).astype(np.uint8))
            # output = Image.fromarray((np.clip(center_crop(output, results[k]['shape'][i]).permute(2,1,0).numpy(),0,1)*255).astype(np.uint8))
            crop = results[k]['cropped'][i].permute(2,1,0)
            detect = (np.clip(results[k]['for_loss'][i].transpose(2,1,0),0,1)*255).astype(np.uint8)

            crop = Image.fromarray(crop.numpy()) #Image.fromarray((np.clip(crop.numpy(),0,1)*255).astype(np.uint8))
            detect = Image.fromarray(detect)

            if not os.path.isdir(os.path.join(save_path, f'{results[k]["phrases"][0][i]}')):
                zz = 1
                os.mkdir(os.path.join(save_path, f'{results[k]["phrases"][0][i]}'))
            save_path = os.path.join(save_path, f'{results[k]["phrases"][0][i]}')
            if os.path.isdir(save_path):
                output.save(os.path.join(save_path, f'output{zz}.png'))
                crop.save(os.path.join(save_path, f'label{zz}.png'))
                detect.save(os.path.join(save_path, f'for_loss{zz}.png'))
                # noc_output.save(os.path.join(save_path, f'noc_output{zz}.png'))
                zz += 1
            

In [24]:
down_scale

8

In [29]:
save_imgs(best_result, f'resultsX{down_scale}x2withDetection_1e-6')

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'resultsX8x2withDetection\\Three stuffed animals are sitting on a bed. \\three stuffed animals'

In [23]:
os.mkdir('./resultsX8x2withDetection\\Three stuffed animals are sitting on a bed.\\three stuffed animals')

'three stuffed animals'

In [31]:
down_scale

8

In [30]:
with open(file=f'resultsX{down_scale}x2withDetection_1e-6.pkl', mode='rb') as f:
    dicdic=pickle.load(f)

In [ ]:
dicdic

In [ ]:
best_result['Three stuffed animals are sitting on a bed. ']["phrases"][0][0]

In [ ]:
best_result['']

In [15]:
import pickle

with open(f'resultsX{down_scale}real_inverse_+_L1_&_L2_not_softmax.pkl','wb') as fw:
    pickle.dump(best_result, fw)

In [16]:
path = './result_model/X8_real_inverse_+_L1_&_L2_not_softmax.pt'
torch.save(best_model, path)

In [ ]:
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

print("PSNR :", peak_signal_noise_ratio(Test_result_img['output'][0], Test_result_img['big_image'][0]))
print("SSIM :", structural_similarity((Test_result_img['output'][0]*255).astype(np.uint8), (Test_result_img['big_image'][0]*255).astype(np.uint8), channel_axis=2,multichannel=True))

In [15]:
nn.Parameter(torch.randn((64))).shape

torch.Size([64])

In [16]:
nn.Parameter(torch.ones((64))).shape

torch.Size([64])

In [59]:
import torch.nn as nn
import torch
sf = nn.Softmax()

a = nn.Parameter(torch.randn((1,1,64,64)))
b = nn.Parameter(torch.randn(1,128,64,64))
print(sf(b).max(dim=1)[0].shape)
print((a.flatten()@b.mean(dim=1).flatten()).shape)


torch.Size([1, 64, 64])
torch.Size([])


C:\Users\juniy\AppData\Local\Temp\ipykernel_14140\2370105043.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(sf(b).max(dim=1)[0].shape)


In [37]:
b.mean(dim=1).shape

torch.Size([1, 64, 64])

In [53]:
torch.dot(a.view(-1,64), b.mean(dim=1).view(-1,64))

RuntimeError: 1D tensors expected, but got 2D and 2D tensors

In [70]:
sf = nn.Softmax(dim=-1)
sf(a@b.mean(dim=1).transpose(-2,-1)).shape

torch.Size([1, 1, 64, 64])